In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalação e atualização do ffmpeg
!apt-get update -y
!apt-get install -y ffmpeg

In [ ]:
import os
import requests
import multiprocessing
from datetime import datetime, timedelta
import json

In [ ]:
# Configuração de diretórios e constantes
OUTPUT_FOLDER = "/content/drive/MyDrive/XCam.Drive/Gravações"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
ABYSS_UPLOAD_URL = 'http://up.hydrax.net/0128263f78f0b426d617bb61c2a8ff43'
RECORD_SECONDS = 180  # 3 minutos para testes

In [ ]:
# Funções utilitárias
def format_seconds(seconds):
    return str(timedelta(seconds=int(seconds)))

def log_progress(username, elapsed_seconds):
    percent = min((elapsed_seconds / RECORD_SECONDS) * 100, 100)
    tempo = format_seconds(elapsed_seconds)
    print(f"⏱️ [{username}] Tempo gravado: {tempo} — 📊 {percent:.1f}% concluído")

In [ ]:
# Etapa 1 – Buscar broadcasts da API XCam (corrigido para estrutura real da resposta e erro de parsing)
def get_broadcasts(limit=5, page=1):
    # Primeira chamada: API principal
    api_url_main = f"https://api.xcam.gay/?limit={limit}&page={page}"
    print(f"🌐 Acessando API principal: {api_url_main}")

    streams_from_main = []
    streams_without_preview = []  # Para armazenar streams que precisam da segunda chamada

    try:
        response_main = requests.get(api_url_main)
        response_main.raise_for_status()
        data_main = response_main.json()

        # --- Debug da Resposta Principal (Mantido para auditoria) ---
        # print("\n--- Resposta completa da API Principal ---")
        # print(json.dumps(data_main, indent=2))
        # print("-----------------------------------------\n")
        # ---------------------------------------------------------

        broadcasts_data = data_main.get("broadcasts")
        if not broadcasts_data:
            print("⚠️ Chave 'broadcasts' não encontrada na resposta da API principal.")
            return []

        items = broadcasts_data.get("items")
        if not isinstance(items, list):
            print(f"⚠️ Chave 'items' não encontrada ou não é uma lista em 'broadcasts' da API principal. Tipo encontrado: {type(items)}")
            return []

        # Processa os items da primeira chamada
        for item in items:
            preview = item.get("preview") or {}
            src = preview.get("src")
            username = item.get("username", "desconhecido")
            if src:
                streams_from_main.append({"username": username, "src": src})
            else:
                print(f"⚠️ Stream sem URL de preview encontrado para usuário (API principal): {username}. Tentando liveInfo...")
                streams_without_preview.append({"username": username})  # Guarda o usuário para a segunda chamada

        print(f"✅ Encontrados {len(streams_from_main)} streams com URL de preview na API principal na página {page}.")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro na requisição HTTP para a API principal: {e}")
        return []
    except json.JSONDecodeError:
        print(f"❌ Erro ao decodificar resposta JSON da API principal. Resposta: {response_main.text}")
        return []
    except Exception as e:
        print(f"❌ Erro inesperado ao acessar API principal: {e}")
        return []

    # Segunda Chamada (Fallback): API liveInfo para streams sem preview
    streams_from_liveinfo = []
    if streams_without_preview:
        print(f"\nAttempting to fetch liveInfo for {len(streams_without_preview)} streams...")  # Log antes da segunda chamada

        for stream_info in streams_without_preview:
            username = stream_info["username"]
            api_url_liveinfo = f"https://api.xcam.gay/user/{username}/liveInfo"
            print(f"🌐 Acessando API liveInfo para {username}: {api_url_liveinfo}")  # Log para ver a URL da segunda chamada

            try:
                response_liveinfo = requests.get(api_url_liveinfo)
                response_liveinfo.raise_for_status()
                data_liveinfo = response_liveinfo.json()

                # --- Debug da Resposta liveInfo (Opcional) ---
                # print(f"\n--- Resposta completa da API liveInfo para {username} ---")
                # print(json.dumps(data_liveinfo, indent=2))
                # print("----------------------------------------------------\n")
                # ------------------------------------------------------------

                # Tenta encontrar a URL m3u8 nas chaves 'cdnURL' ou 'edgeURL'
                m3u8_url = data_liveinfo.get("cdnURL") or data_liveinfo.get("edgeURL")

                if m3u8_url:
                    print(f"✅ URL .m3u8 encontrada em liveInfo para {username}.")
                    streams_from_liveinfo.append({"username": username, "src": m3u8_url})
                else:
                    print(f"⚠️ Nenhuma URL .m3u8 válida encontrada em liveInfo para o usuário: {username}.")

                # Adiciona um pequeno delay para não sobrecarregar a segunda API
                time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"❌ Erro na requisição HTTP para a API liveInfo de {username}: {e}")
            except json.JSONDecodeError:
                print(f"❌ Erro ao decodificar resposta JSON da API liveInfo de {username}. Resposta: {response_liveinfo.text}")
            except Exception as e:
                print(f"❌ Erro inesperado ao acessar API liveInfo de {username}: {e}")

    # Combina os resultados das duas chamadas
    final_streams_list = streams_from_main + streams_from_liveinfo
    print(f"\nFinal list for page {page}: Found a total of {len(final_streams_list)} streams with valid URLs after fallback.")

    return final_streams_list

In [ ]:
# Etapa 2 – Gravar stream com ffmpeg por 3 minutos (teste)
def gravar_stream(username, m3u8_url):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{username}_{timestamp}.mp4"
    filepath = os.path.join(OUTPUT_FOLDER, filename)
    print(f"\n🎬 Iniciando gravação de: {username} (URL: {m3u8_url})")  # Log mais detalhado
    # Limite de tempo ajustado para 3 minutos (00:03:00)
    # Uso de subprocess.run é geralmente preferível a os.system para maior controle
    import subprocess
    ffmpeg_cmd = ["ffmpeg", "-i", m3u8_url, "-t", "00:03:00", "-c", "copy", "-y", filepath]

    start_time = datetime.now()
    try:
        # Adicionado captura de saída e erro para diagnóstico
        process = subprocess.run(ffmpeg_cmd, capture_output=True, text=True, check=False)
        elapsed_seconds = (datetime.now() - start_time).total_seconds()
        log_progress(username, elapsed_seconds)

        if process.returncode != 0:
            print(f"❌ FFmpeg falhou para {username}. Código de saída: {process.returncode}")
            print(f"   Stderr: {process.stderr}")
            # Não prossegue com upload se a gravação falhou
            return {
                'username': username,
                'filename': filename,
                'filepath': filepath,
                'upload_success': False,
                'abyss_response': "Gravação FFmpeg falhou"
            }
        else:
            print(f"✅ Gravação finalizada com sucesso: {filename}")
            success, abyss_resp = upload_to_abyss(filepath)
            return {
                'username': username,
                'filename': filename,
                'filepath': filepath,
                'upload_success': success,
                'abyss_response': abyss_resp
            }
    except FileNotFoundError:
        print(f"❌ Erro: Comando 'ffmpeg' não encontrado. Certifique-se de que foi instalado corretamente.")
        return {
            'username': username,
            'filename': filename,
            'filepath': filepath,
            'upload_success': False,
            'abyss_response': "Comando FFmpeg não encontrado"
        }
    except Exception as e:
        print(f"❌ Erro inesperado durante a gravação FFmpeg para {username}: {e}")
        return {
            'username': username,
            'filename': filename,
            'filepath': filepath,
            'upload_success': False,
            'abyss_response': f"Erro inesperado na gravação: {e}"
        }

In [ ]:
# Etapa 3 – Upload para Abyss.to com tratamento robusto
def upload_to_abyss(filepath):
    file_name = os.path.basename(filepath)
    file_type = 'video/mp4'
    print(f"⬆️ Tentando upload de: {file_name} para Abyss.to...") # Log de início de upload
    try:
        with open(filepath, 'rb') as f:
            files = { 'file': (file_name, f, file_type) }
            response = requests.post(ABYSS_UPLOAD_URL, files=files)

            # Log do status code da resposta do Abyss
            print(f"   Status code da resposta do Abyss.to: {response.status_code}")

            try:
                resp_json = response.json()
                # Log da resposta JSON completa para auditoria
                print(f"   Resposta JSON do Abyss.to: {json.dumps(resp_json, indent=2)}")

                if resp_json.get('status'):
                    uploaded_url = resp_json.get('urlIframe')
                    if uploaded_url:
                        print(f"📤 Upload realizado com sucesso! URL: {uploaded_url}")
                        return True, resp_json # Retorna True e a resposta JSON completa
                    else:
                        print(f"✅ Upload realizado com sucesso, mas URL não retornada na resposta JSON.")
                        # Ainda consideramos sucesso no upload se status é True, mas alertamos sobre a falta da URL
                        return True, resp_json # Ainda retorna True e a resposta JSON completa
                else:
                    # O status não é true, indica falha no upload
                    error_message = resp_json.get('message', 'Mensagem de erro não fornecida')
                    print(f"❌ Falha no upload para Abyss.to. Status 'status': false. Mensagem: {error_message}")
                    return False, resp_json # Retorna False e a resposta JSON completa

            except json.JSONDecodeError:
                print(f"⚠️ Resposta do Abyss.to não é JSON. Status: {response.status_code}. Resposta: {response.text}") # Log de resposta não JSON
                return False, response.text # Retorna False e a resposta de texto completa

            except Exception as e:
                 print(f"⚠️ Erro inesperado ao processar resposta JSON do Abyss.to: {e}. Resposta: {response.text}") # Log de erro inesperado no JSON
                 return False, response.text # Retorna False e a resposta de texto completa

    except FileNotFoundError:
        print(f"❌ Erro: Arquivo não encontrado para upload: {filepath}") # Log de arquivo não encontrado
        return False, "Arquivo não encontrado" # Retorna False e uma string de erro
    except Exception as e:
        print(f"❌ Erro geral no upload: {e}") # Log de erro geral
        return False, str(e) # Retorna False e a string do erro

In [ ]:
# Etapa 4 – Processamento paginado e execução paralela
def process_page(page=1, limit=5):
    print(f"\n📄 Iniciando processamento da página {page}\n")
    streams = get_broadcasts(limit=limit, page=page)

    # A condição abaixo será ativada se get_broadcasts retornar uma lista vazia
    if not streams:
        print(f"\n🚫 Nenhuma stream encontrada com URL válida na página {page}. Encerrando.")
        return False

    jobs = []
    # O Manager().list() é essencial para compartilhar a lista entre processos
    results = multiprocessing.Manager().list()

    # Função worker que será executada em cada processo
    def worker(username, m3u8_url, results):
        result = gravar_stream(username, m3u8_url)
        results.append(result)

    print(f"🚀 Iniciando gravação paralela para {len(streams)} streams...")
    for stream in streams:
        username = stream["username"]
        m3u8_url = stream["src"]
        # Cria um novo processo para cada stream
        p = multiprocessing.Process(target=worker, args=(username, m3u8_url, results))
        jobs.append(p)
        p.start() # Inicia o processo

    # Espera todos os processos terminarem
    for job in jobs:
        job.join()

    print(f"\n🏁 Todas as gravações e uploads da página {page} foram concluídos.")
    # Opcional: Processar ou exibir os resultados coletados em 'results'
    # print("\nResumo dos resultados:")
    # for res in results:
    #     print(f"  - Usuário: {res['username']}, Sucesso Upload: {res['upload_success']}")

    return True # Retorna True se encontrou streams para processar nesta página

def main():
    page = 1
    limit = 5  # Você pode ajustar o limite para testar com mais ou menos streams por página
    print("🤖 Iniciando o processo principal de busca e gravação de streams...")
    while True:
        # Processa uma página de streams
        ok = process_page(page=page, limit=limit)

        # Se process_page retornar False, significa que não encontrou streams na página
        if not ok:
            break # Sai do loop

        page += 1 # Avança para a próxima página
        # Opcional: Adicionar um pequeno delay entre as páginas para não sobrecarregar a API
        # import time
        # time.sleep(5)

    print("\n✨ Processo principal concluído.")

# Para rodar no Colab
if __name__ == '__main__':
    # Verifica se está rodando no Colab para evitar problemas de multiprocessing em outros ambientes
    try:
        if 'google.colab' in str(get_ipython()):
            main()
        else:
            print("Script não está rodando no Google Colab. Execute main() manualmente se desejar.")
            # Você pode chamar main() aqui se souber que o ambiente suporta multiprocessing como esperado
            # main()
    except NameError:
        print("Não está rodando em um ambiente IPython/Colab. Execute main() manualmente se desejar.")
        # main()